# Preprocessing

Voici les colonnes gardées pour la fin de la préparation des données :

- `price`: moyenne entre le `high` et `low` de chaque ligne
- `volume`: donnée de volume du dataset de base
- `trend`: évolution du cours entre maintenant et t-5h
- `quadratic_variation`: cette colonne représente la variation quadratique du cours de l'action au cours des 20 dernières minutes. Elle est calculé en se basant sur les high et low de chaque minute

import packages


In [72]:
import pandas as pd
import numpy as np

# Import data

In [108]:
path = "BTCUSDT-minutes.csv"
data = pd.read_csv(path)

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57601 entries, 0 to 57600
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  57601 non-null  object 
 1   open       57601 non-null  float64
 2   high       57601 non-null  float64
 3   low        57601 non-null  float64
 4   close      57601 non-null  float64
 5   volume     57601 non-null  float64
dtypes: float64(5), object(1)
memory usage: 2.6+ MB


In [110]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [111]:
data.head(5)

,timestamp,open,high,low,close,volume
0,2023-02-01 00:00:00,23125.13,23140.47,23124.41,23133.74,199.09155
1,2023-02-01 00:01:00,23133.74,23137.00,23123.15,23123.24,153.92866
2,2023-02-01 00:02:00,23123.16,23145.99,23123.01,23142.98,204.22769
3,2023-02-01 00:03:00,23144.00,23168.90,23141.75,23157.56,385.73800
4,2023-02-01 00:04:00,23158.68,23159.26,23131.97,23133.11,209.27799


# Data Engineering

In [112]:
data["price"] = (data["high"] + data["low"])/2
data.set_index('timestamp', inplace=True, drop=False)
data.head(5)

,timestamp,open,high,low,close,volume,price
timestamp,,,,,,,
2023-02-01 00:00:00,2023-02-01 00:00:00,23125.13,23140.47,23124.41,23133.74,199.09155,23132.440
2023-02-01 00:01:00,2023-02-01 00:01:00,23133.74,23137.00,23123.15,23123.24,153.92866,23130.075
2023-02-01 00:02:00,2023-02-01 00:02:00,23123.16,23145.99,23123.01,23142.98,204.22769,23134.500
2023-02-01 00:03:00,2023-02-01 00:03:00,23144.00,23168.90,23141.75,23157.56,385.73800,23155.325
2023-02-01 00:04:00,2023-02-01 00:04:00,23158.68,23159.26,23131.97,23133.11,209.27799,23145.615


In [113]:
# Calcul de l'écart au carré max chaque minute
data["value_var"] = (data["high"] - data["low"])**2
# Réalisation d'une moyenne pour obtenir la variance et ainsi la valeur souhaitée
data["quadratic_variation"] = data["value_var"].rolling("20min", center=False).mean()

In [114]:
data["trend"] =  (data['price'] - data['price'].shift(300))/data['price']

In [115]:
data.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'price',
       'value_var', 'quadratic_variation', 'trend'],
      dtype='object')

In [116]:
column_to_keep = ['timestamp','volume', 'price', 'quadratic_variation', 'trend']
data = data[column_to_keep]
data

,timestamp,volume,price,quadratic_variation,trend
timestamp,,,,,
2023-02-01 00:00:00,2023-02-01 00:00:00,199.09155,23132.440,257.923600,NaN
2023-02-01 00:01:00,2023-02-01 00:01:00,153.92866,23130.075,224.873050,NaN
2023-02-01 00:02:00,2023-02-01 00:02:00,204.22769,23134.500,325.942167,NaN
2023-02-01 00:03:00,2023-02-01 00:03:00,385.73800,23155.325,428.737250,NaN
2023-02-01 00:04:00,2023-02-01 00:04:00,209.27799,23145.615,491.938620,NaN
...,...,...,...,...,...
2023-03-12 23:56:00,2023-03-12 23:56:00,703.64043,22010.200,6709.428905,0.039253
2023-03-12 23:57:00,2023-03-12 23:57:00,615.34747,22019.410,6697.022120,0.039102
2023-03-12 23:58:00,2023-03-12 23:58:00,538.69132,22002.065,6727.026920,0.038486


# Exportation en csv

In [117]:
data.to_csv('btc_clean.csv', index=False)